#OR Final Project - Mahya Shahshahani SID:810199598

# Problem 1

In [52]:
!pip install pyomo
from pyomo.environ import *
!wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64
!apt-get install -y -qq glpk-utils


In [53]:
from pyomo.opt import SolverFactory
!apt-get install -y gcc libglpk-dev glpk-utils
import heapq

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
gcc is already the newest version (4:11.2.0-1ubuntu1).
glpk-utils is already the newest version (5.0-1).
libglpk-dev is already the newest version (5.0-1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [54]:
!apt install glpk-utils
!pip install glpk


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
glpk-utils is already the newest version (5.0-1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [55]:
import pyomo.environ as pyo
from pyomo.environ import *
!pip install glpk
from pyomo.opt import SolverFactory


model = pyo.ConcreteModel()

model.x = pyo.Var([1, 2, 3, 4, 5, 6], [1, 2], domain=NonNegativeReals)

model.S = Param([1, 2], initialize={1: 550, 2: 700})
model.D = Param([1, 2, 3, 4, 5, 6], initialize={1: 125, 2: 175, 3: 225, 4: 250, 5: 225, 6: 200})
model.T = Param([1, 2, 3, 4, 5, 6], [1, 2], initialize={
    (1, 1): 9999, (1, 2): 2.5,
    (2, 1): 2.5, (2, 2): 9999,
    (3, 1): 1.6, (3, 2): 2.0,
    (4, 1): 1.4, (4, 2): 1.0,
    (5, 1): 0.8, (5, 2): 1.0,
    (6, 1): 1.4, (6, 2): 0.8
})

model.objective = Objective(
    expr=sum(model.T[c, s] * model.x[c, s] for c in [1, 2, 3, 4, 5, 6] for s in [1, 2]),
    sense=minimize
)


model.supply_constraints = ConstraintList()
for s in [1, 2]:
    model.supply_constraints.add(
        sum(model.x[c, s] for c in [1, 2, 3, 4, 5, 6]) <= model.S[s]
    )

model.demand_constraints = ConstraintList()
for c in [1, 2, 3, 4, 5, 6]:
    model.demand_constraints.add(
        sum(model.x[c, s] for s in [1, 2]) == model.D[c]
    )

solver = SolverFactory('glpk')
solver.solve(model)

for c in [1, 2, 3, 4, 5, 6]:
    for s in [1, 2]:
        print(f"City {c} from Supply {s}: {model.x[c, s].value}")

City 1 from Supply 1: 0.0
City 1 from Supply 2: 125.0
City 2 from Supply 1: 175.0
City 2 from Supply 2: 0.0
City 3 from Supply 1: 225.0
City 3 from Supply 2: 0.0
City 4 from Supply 1: 0.0
City 4 from Supply 2: 250.0
City 5 from Supply 1: 150.0
City 5 from Supply 2: 75.0
City 6 from Supply 1: 0.0
City 6 from Supply 2: 200.0


In [56]:
import pyomo.environ as pyo
from pyomo.environ import *
!pip install glpk
from pyomo.opt import SolverFactory


model = pyo.ConcreteModel()


model.x = pyo.Var([1, 2, 3, 4, 5, 6], [1, 2], domain=NonNegativeReals)

model.S = Param([1, 2], initialize={1: 600, 2: 650})
model.D = Param([1, 2, 3, 4, 5, 6], initialize={1: 125, 2: 175, 3: 225, 4: 250, 5: 225, 6: 200})
model.T = Param([1, 2, 3, 4, 5, 6], [1, 2], initialize={
    (1, 1): 9999, (1, 2): 2.5,
    (2, 1): 2.5, (2, 2): 9999,
    (3, 1): 1.6, (3, 2): 2.0,
    (4, 1): 1.4, (4, 2): 1.0,
    (5, 1): 0.8, (5, 2): 1.0,
    (6, 1): 1.4, (6, 2): 0.8
})


model.objective = Objective(
    expr=sum(model.T[c, s] * model.x[c, s] for c in [1, 2, 3, 4, 5, 6] for s in [1, 2]),
    sense=minimize
)


model.supply_constraints = ConstraintList()
for s in [1, 2]:
    model.supply_constraints.add(
        sum(model.x[c, s] for c in [1, 2, 3, 4, 5, 6]) <= model.S[s]
    )

model.demand_constraints = ConstraintList()
for c in [1, 2, 3, 4, 5, 6]:
    model.demand_constraints.add(
        sum(model.x[c, s] for s in [1, 2]) == model.D[c]
    )


solver = SolverFactory('glpk')
solver.solve(model)

for c in [1, 2, 3, 4, 5, 6]:
    for s in [1, 2]:
        print(f"City {c} from Supply {s}: {model.x[c, s].value}")

City 1 from Supply 1: 0.0
City 1 from Supply 2: 125.0
City 2 from Supply 1: 175.0
City 2 from Supply 2: 0.0
City 3 from Supply 1: 225.0
City 3 from Supply 2: 0.0
City 4 from Supply 1: 0.0
City 4 from Supply 2: 250.0
City 5 from Supply 1: 200.0
City 5 from Supply 2: 25.0
City 6 from Supply 1: 0.0
City 6 from Supply 2: 200.0


#Problem 2


In [57]:
import numpy as np

def objective(x, Q, q, p):
    return 0.5 * x.T.dot(Q).dot(x) - q.T.dot(x) + p

def gradient(x, Q, q):
    return Q.dot(x) - q

def gradient_descent(Q, q, p, x0, learning_rate=0.1, epsilon=1e-6, max_iterations=1000):
    x = x0.astype(np.float64)
    for i in range(max_iterations):
        grad = gradient(x, Q, q)
        step_size = learning_rate / np.sqrt(i + 1)  # Optimized step size
        x = x - (step_size * grad).astype(np.float64)
        if np.linalg.norm(grad) < epsilon:
            break
    return x.astype(np.int64)
Q = np.array([[48, 12], [8, 8]])
q = np.array([[13], [23]])
p = 4
x0 = np.array([[23], [37]])

optimized_x = gradient_descent(Q, q, p, x0)
print("Optimized x:")
print(optimized_x)

Optimized x:
[[0]
 [3]]


In [58]:
import numpy as np

def objective(x, Q, q, p):
    return 0.5 * x.T.dot(Q).dot(x) - q.T.dot(x) + p

def gradient(x, Q, q):
    return Q.dot(x) - q

def gradient_descent(Q, q, p, x0, learning_rate, epsilon=1e-6, max_iterations=1000):
    x = x0.astype(np.float64)
    for i in range(max_iterations):
        grad = gradient(x, Q, q)
        x = x - learning_rate * grad
        if np.linalg.norm(grad) < epsilon:
            break
    return x.astype(np.int64)

Q = np.array([[48, 12], [8, 8]])
q = np.array([[13], [23]])
p = 4
x0 = np.array([[23], [37]])

optimized_x_0001 = gradient_descent(Q, q, p, x0, 0.0001)
print("Optimized x learning rate = 0.0001:")
print(optimized_x_0001)

optimized_x_001 = gradient_descent(Q, q, p, x0, 0.01)
print("Optimized x learning rate = 0.01:")
print(optimized_x_001)

optimized_x_003 = gradient_descent(Q, q, p, x0, 0.03)
print("Optimized x learning rate = 0.03:")
print(optimized_x_003)

optimized_x_004 = gradient_descent(Q, q, p, x0, 0.04)
print("Optimized x learning rate = 0.04:")
print(optimized_x_004)

optimized_x_005 = gradient_descent(Q, q, p, x0, 0.05)
print("Optimized x learning rate = 0.05:")
print(optimized_x_005)


Optimized x learning rate = 0.0001:
[[-4]
 [19]]
Optimized x learning rate = 0.01:
[[0]
 [3]]
Optimized x learning rate = 0.03:
[[0]
 [3]]
Optimized x learning rate = 0.04:
[[1516021]
 [ 286917]]
Optimized x learning rate = 0.05:
[[-9223372036854775808]
 [-9223372036854775808]]


##Problem 3

In [59]:
def optimal_path(graph, start, destination, epsilon = 0.5):

    distances = {node: float('inf') for node in graph}
    distances[start] = 0
    visited = {}


    pq = [(0, start)]

    while pq:
        current_dist, current_node = heapq.heappop(pq)

        # Skip the visited nodes
        if current_node in visited:
            continue

        visited[current_node] = True

        # Check if the destination node is reached
        if current_node == destination:
            break

        # Update distances to neighbors
        for neighbor, edge_data in graph[current_node].items():
            new_dist = current_dist + edge_data['cost']
            if new_dist < distances[neighbor]:
                distances[neighbor] = new_dist
                heapq.heappush(pq, (new_dist, neighbor))

    path = []
    current = destination
    while current != start:
        path.append(current)
        current = min(graph[current], key=lambda x: distances[x] + epsilon * graph[current][x]['cost'])
    path.append(start)
    path.reverse()

    return path, distances[destination]

graph = {
    '0': {'1': {'cost': 8.28}, '2': {'cost': 1.28}, '6': {'cost': 4.48}},
    '1': {'0': {'cost': 3.64}, '5': {'cost': 7.52}, '7': {'cost': 7.56}},
    '2': {'0': {'cost': 0.66}},
    '3': {'9': {'cost': 10}},
    '4': {'5': {'cost': 19.38}, '8': {'cost': 44}, '9': {'cost': 42}},
    '5': {'1': {'cost': 11.38}, '4': {'cost': 24.96}, '7': {'cost': 10.44}},
    '6': {'0': {'cost': 1.6}, '7': {'cost': 11.4}},
    '7': {'1': {'cost': 9.12}, '5': {'cost': 9.12}, '6': {'cost': 8.4}},
    '8': {'4': {'cost': 37.6}, '9': {'cost': 29.58}},
    '9': {'3': {'cost': 4.32}, '4': {'cost': 36.12}, '8': {'cost': 28.6}},
}

locations = {
    '0': (35.702, 51.396),
    '1': (35.721, 51.388),
    '2': (35.701, 51.39),
    '3': (35.784, 51.398),
    '4': (35.753, 51.426),
    '5': (35.742, 51.399),
    '6': (35.703, 51.41),
    '7': (35.728, 51.417),
    '8': (35.79, 51.457),
    '9': (35.777, 51.41),
}

#Input
start_point = '0'
destination_point = '3'

best_path, minimal_cost = optimal_path(graph, start_point, destination_point)

# Desired Output
print("start point:", start_point, locations[start_point])
print("destination point:", destination_point,locations[destination_point])
print("Optimal Path:", best_path)
print("Minimal Cost:", minimal_cost)


start point: 0 (35.702, 51.396)
destination point: 3 (35.784, 51.398)
Optimal Path: ['0', '1', '5', '4', '9', '3']
Minimal Cost: 87.07999999999998
